# Crowd dynamics simulation — a multi-agent system based on CA (16 points)

Today you'll create a more complex simulation of crowd dynamics. We are trying to model the movement of people trying to leave a room (e.g. lectureroom, stadium etc.). The room has one or more exit and people insidie want to leave it using the closest exit. The model uses static field which can be treated as a *distance* to the closest exit. Agents move toward the exits by choosing the cells with the smallest value of static field in their neighbourhoods. Static field is also used to simulate walls (wall is a cell with ridiculously high static field value, so no agent will choose it).


## (2 points) Neighbourhood initialisation — `Board` Class
Fill in the missing part of the `__init__` method in the `Board` class. Use Moore neighbourhood. **Do not initialise neighbourhoods for border cells.**
![image](https://upload.wikimedia.org/wikipedia/commons/8/86/CA-Moore.svg)

<center>Moore neighbourhood (source: <a href="https://commons.wikimedia.org/wiki/File:CA-Moore.svg">wikimedia.org</a>)</center>

## (3 points) Static potential field

### Implement `calculateField` method in the `Board` class:
* Create a list of points (`toCheck`) for which the static field should be recalculated (initially each cell's `staticField` is equal to `100000`). 
* Change `staticField` of all exits (points with `pointType == 2`) to `0`. 
* Add all exits' neighbours to the `toCheck` list.
* Until the `toCheck` list is empty:
    * check if the `staticField` of the first element of the list has changed (to do so call its `calcStaticField()` method).
    * If the value changed, add all neighbours of this cell to the end of the `toCheck` list.
    * Remove the first element from the list.

### Implement `calcStaticField` method in the `Point` class:
* Find the smallest `staticField` in the cell's neighbourhood.
* If the cell's `staticField` is greater than the found value + 1 (`self.staticField > neighbourMin + 1`), change the `staticField` to this value (i.e. `neighbourMin + 1`) and return `True`, otherwise do not change anything and return `False`. **Do not change walls static fields**.

## (3 points) Implement `createBoard` method in the `Board` class:
* (1 point) Add some walls and pedestrians, add at least one exit.
* (2 points) Create a random version of this method (call it `randomBoard`), try to generate walls in patterns.

## Naive Implementation (6 points)

### Implement `move` method in the `Point` class:
* If the cell represents a pedestrian (`pointType == 3`), move the pedestrian to the neighbouring cell with the smallest `staticField`.
* Run the simulation and observe what is happening. 
* Is everything working as it should? If not, what's wrong? How can we fix it?

## First improvements (4 points)
There are two main reasons responsible for the visible errors:
* There is no *exit* mechanism. After reaching the exit, the agent should be removed from the `Board`.
* No cells synchronisation. One should note that agents moving down and right can reach the destination in one iteration. To fix this issue you can add a boolean value `isBlocked = false` to the `Point` class. After the agent moves, the `isBlocked` value of the occupied cell should be change to `True`. Remember to *unblock* all cells at the beginning of each iteration. 

Correct the errors and create a complex board with walls, many pedestrians and at least two exits. Check the behaviour of agents. Is everything right? Any other problems?





In [ ]:
import numpy as np
import itertools
import pygame
import random
import copy

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

## `Point` class

In [ ]:
class Point:
    def __init__(self):
        # 0 - floor, 1 - wall, 2 - exit, 3 - pedestrian
        self.pointType = 0
        self.staticField = 100000
        self.neighbours = []

    def clear(self):
        self.staticField = 100000


    def calcStaticField(self):
        # TODO
        return False

    def move(self):
        # TODO
             
        return


    def getColour(self):
        if self.pointType == 0:
            # Floor
            return (243, 243, 243)
        elif self.pointType == 1:
            # Wall
            return (255, 191, 0)
        elif self.pointType == 2:
            # Exit
            return (0, 255, 56)
        else:
            # Pedestrian
            return (0, 39, 255)
        


# `Board` class

In [ ]:
class Board:
    
    def __init__(self, xSize = 40, ySize = 40):
        
        # Create empty board 
        self.points = [[Point() for i in range(0, ySize)] for j in range(0, xSize)]
        
        # Initialise neighbourhood
        # TODO
        
                                                    
    def calculateField(self):
        # TODO 
        return
    
    def createBoard(self):
        # TODO
        # 0 - floor, 1 - wall, 2 - exit, 3 - pedestrian
        return

        
    def iteration(self):                
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].move()

    
    def clear(self):
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].clear()
        self.calculateField()
    
    # Visualisation
    def drawGrid(self, screen, w_width, w_height):
        rows = len(self.points)
        cols = len(self.points[0])
        blockSize = (min(w_width, w_height)-max(rows, cols))/max(rows, cols)

        for x in range(0, rows):
            for y in range(0, cols):
                pos_x = (blockSize+1) * x
                pos_y = (blockSize+1) * y
                rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
                pygame.draw.rect(screen, self.points[x][y].getColour(), rect, 0)    
        pygame.display.flip()
        
    
    # For debug, get numpy arrays of field and 
        
    def getFloorFieldValues(self):
        # Returns numpy array of fieldValues
        fig = np.zeros((len(self.points),len(self.points[0])))
        for i in range(0,len(self.points)):
            for j in range(0, len(self.points[0])):
                fig[i][j] = self.points[i][j].staticField
        return fig
    
    def getPointTypes(self):
        # Returns numpy array of colours
        fig = np.zeros((len(self.points),len(self.points[0])))
        for i in range(0,len(self.points)):
            for j in range(0, len(self.points[0])):
                fig[i][j] = self.points[i][j].pointType
        return fig

# `Pygame` stuff

In [ ]:
pygame.init()
# Parameters
w_width = 800
w_height = 800
# Set up the drawing window, adjust the size
screen = pygame.display.set_mode([w_width, w_height])

# Create Board object
board = Board()

# Set background
screen.fill((255, 255, 255))

# Add walls, exits & humans
board.createBoard()
# Calculate static fields
board.calculateField()

# Draw grid
board.drawGrid(screen, w_width, w_height)
    
running = True

time_delay = 200 # 0.2 s
timer_event = pygame.USEREVENT + 1
pygame.time.set_timer(timer_event, time_delay )

while running:
    for event in pygame.event.get():   
        if event.type == QUIT:
            running = False
        
        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                board.iteration()
                board.drawGrid(screen, w_width, w_height)

        if event.type == timer_event:
            board.iteration()
            board.drawGrid(screen, w_width, w_height)

pygame.quit()

In [ ]:
# print board
np.set_printoptions(suppress=True)
with np.printoptions(threshold=np.inf):
    print(board.getPointTypes())

In [ ]:
# print static field values
np.set_printoptions(suppress=True)
with np.printoptions(threshold=np.inf):
    print(board.getFloorFieldValues())